# Tarea 4

### Indicaciones

Para esta tarea cuenta con dos bases de datos, una de clientes y otra de compras.

La base de datos de clientes tiene la siguiente estructura:
- `id_cliente`: identificador único del cliente.
- `nombre`: nombre del cliente.
- `direccion`: dirección del cliente.
- `email`: correo electrónico del cliente.
- `edad`: edad del cliente.
- `fecha_registro`: fecha en la que el cliente se registró.
- `activo`: indica si el cliente está activo o no.

La base de datos de compras tiene la siguiente estructura:
- `id_compra`: identificador único de la compra.
- `id_cliente`: identificador único del cliente que realizó la compra.
- `producto`: producto comprado.
- `cantidad`: cantidad de productos comprados.
- `precio`: precio del producto.
- `fecha_compra`: fecha en la que se realizó la compra.

Usted deberá realizar diferentes acciones sobre estas bases de datos.

## 1. Limpieza de datos

### Clientes (4 puntos)
1. Elimine los registros de clientes que no tengan un email válido o que no tengan email (puede usar expresiones regulares). (0.5 puntos)
2. Hay registros repetidos en la base de datos de clientes, elimine los registros repetidos. (0.5 puntos)
3. Hay registros con edades negativas, reemplace estas edades por la mediana de las edades de los clientes. (1 punto)
4. Hay registros sin nombre, reemplace estos nombres por "Cliente sin nombre". (0.5 puntos)
5. Hay registros sin dirección, reemplace estas direcciones por "Dirección desconocida". (0.5 puntos)
6. Hay clientes que no indican si están activos o no, reemplace estos valores por False. (1 punto)

### Compras (3 puntos)
1. Hay registros con cantidades negativas, elimine estos registros. (0.5 puntos)
2. Hay registros con precios de tipo string en vez de numéricos, elimine estos registros. (0.5 puntos)
3. Hay registros con productos vacíos, reemplácelos por "Producto desconocido". (0.5 puntos)
4. Hay registros con `id_cliente` que no existen en la base de datos de clientes, elimine estos registros. (1 punto)
5. Hay registros con `id_cliente` vacíos, elimine estos registros. (0.5 puntos)

In [6]:
# Limpieza de clientes

import pandas as pd
import numpy as np
from datetime import datetime

# Cargar datos
clientes_df = pd.read_csv('/content/clientes.csv')

# 1. Eliminar registros sin email válido o sin email
clientes_df = clientes_df[clientes_df['email'].str.contains('^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$', na=False)]

# 2. Eliminar registros duplicados
clientes_df.drop_duplicates(inplace=True)

# 3. Reemplazar edades negativas por la mediana de las edades
median_age = clientes_df[clientes_df['edad'] > 0]['edad'].median()
clientes_df['edad'] = clientes_df['edad'].apply(lambda x: median_age if x < 0 else x)

# 4. Reemplazar nombres faltantes
clientes_df['nombre'].replace(np.nan, 'Cliente sin nombre', inplace=True)

# 5. Reemplazar direcciones faltantes
clientes_df['direccion'].replace(np.nan, 'Dirección desconocida', inplace=True)

# 6. Reemplazar valores faltantes en "activo"
clientes_df['activo'].replace(np.nan, False, inplace=True)
clientes_df['activo'] = clientes_df['activo'].astype(bool)  # Asegurar que todos los valores sean booleanos

# Mostrar los datos limpios
clientes_df.head()


,id_cliente,nombre,direccion,email,edad,fecha_registro,activo
0,bdd640fb-0667-4ad1-9c80-317fa3b1799d,Daniel Doyle,"600 Jeffery Parkways\nNew Jamesside, MT 29394",zlawrence@example.org,56.0,2023-03-23,False
1,571aa876-6c30-4511-b2b9-437a28df6ec4,Daniel Gallagher,"55940 Herrera Lane Apt. 849\nBarbaraland, AZ 8...",kendragalloway@example.org,46.0,2023-10-06,True
2,a2bc372f-7412-4293-8729-4739614ff3d7,Carla Kelly,"283 Steven Groves\nLake Mark, WI 07832",lydiatrujillo@example.com,60.0,2020-10-07,False
3,9132b63e-f162-47e4-a9c3-49e03602f8ac,Emily Rios,"42388 Burgess Meadow Suite 532\nDaviston, VI 1...",courtneyconner@example.com,78.0,2022-05-30,True
4,ff01cf99-988c-44c9-a1b1-cd2262801c45,David Tran,USS Maldonado\nFPO AP 98920,kayla51@example.com,56.0,2021-08-08,False


In [7]:
# Limpieza de compras

# Cargar datos
compras_df = pd.read_csv('/content/compras.csv')

# 1. Eliminar registros con cantidades negativas
compras_df = compras_df[compras_df['cantidad'] >= 0]

# 2. Eliminar registros con precios no numéricos
compras_df['precio'] = pd.to_numeric(compras_df['precio'], errors='coerce')
compras_df = compras_df.dropna(subset=['precio'])

# 3. Reemplazar productos faltantes
compras_df['producto'].replace('', 'Producto desconocido', inplace=True)
compras_df['producto'].replace(pd.NA, 'Producto desconocido', inplace=True)

# 4. Eliminar registros con `id_cliente` que no exista en la base de clientes (esto requiere datos de clientes)
# clientes_df ya debe estar cargado y limpio
clientes_validos = set(clientes_df['id_cliente'])
compras_df = compras_df[compras_df['id_cliente'].isin(clientes_validos)]

# 5. Eliminar registros con `id_cliente` vacío
compras_df = compras_df[compras_df['id_cliente'].notna()]

# Mostrar los datos limpios
compras_df.head()

,id_compra,id_cliente,producto,cantidad,precio,fecha_compra
0,b1fead13-f280-4f1d-9a6c-0db952171c88,f94d8fcb-dba3-496f-ba32-3f6990dbfad6,deep,4,454.25,2023-05-22
1,f7fe604f-65f3-4779-b6ee-5cdb2d79a2d1,fff9f585-0d55-4b61-8a17-5dfebfc00dc8,from,7,208.14,2023-01-05
2,f7bf5244-7db2-47aa-a205-e54fb1e8c09d,a6348e78-4d5c-45c7-b379-cb1ee8cda0cc,parent,4,471.71,2023-12-29
3,d105c8d6-47c1-4017-a045-2909536d47d7,39b86bb2-b596-4a7c-af4a-fa885299e505,stock,19,84.80,2023-03-20
4,33261887-9058-41df-ae9b-f3e11d27c09a,b0946d2a-2aa9-4b43-ad15-f16fb7b8c1a5,person,10,405.02,2024-06-04


## 2. Stats

Calcular la media, mediana y skew de las edades y precios usando `.agg`. (2 puntos)

In [8]:
# Edades

# Calcular media, mediana y skew de las edades
resultados_edades = clientes_df['edad'].agg(['mean', 'median', 'skew'])

# Mostrar los resultados
print(resultados_edades)

mean      48.171717
median    46.000000
skew       0.098407
Name: edad, dtype: float64


In [9]:
# Precios

# Calcular media, mediana y skew de los precios
resultados_precios = compras_df['precio'].agg(['mean', 'median', 'skew'])

# Mostrar los resultados
print(resultados_precios)

mean      264.516211
median    257.405000
skew       -0.014805
Name: precio, dtype: float64


## 3. Concatenación

Halle las compras desde el 2023 hasta hoy y guárdelas en una variable llamada `compras_2023_hoy`. (1 punto)

Halle las compras de diciembre de 2022 y guárdelas en una variable llamada `compras_2022_navidad`. (1 punto)

Concatene ambos dataframes y guarde el resultado en una variable llamada `compras_concatenadas`. (1 punto)

In [10]:
# Compras de 2023 a hoy

# Convertir la columna 'fecha_compra' a tipo datetime si aún no se ha hecho
compras_df['fecha_compra'] = pd.to_datetime(compras_df['fecha_compra'])

# Filtrar compras desde el inicio de 2023 hasta hoy
fecha_inicio = pd.Timestamp(year=2023, month=1, day=1)
fecha_fin = pd.Timestamp(datetime.now())  # Hoy

compras_2023_hoy = compras_df[(compras_df['fecha_compra'] >= fecha_inicio) & (compras_df['fecha_compra'] <= fecha_fin)]

# Mostrar el resultado
print(compras_2023_hoy)

                                id_compra  \
0    b1fead13-f280-4f1d-9a6c-0db952171c88   
1    f7fe604f-65f3-4779-b6ee-5cdb2d79a2d1   
2    f7bf5244-7db2-47aa-a205-e54fb1e8c09d   
3    d105c8d6-47c1-4017-a045-2909536d47d7   
4    33261887-9058-41df-ae9b-f3e11d27c09a   
..                                    ...   
193  9d1c0838-edc8-4ca8-a3bc-d6f3b18d2c8c   
194  2ce3d822-d080-42b8-96c5-6fe7aa604892   
196  643ffdae-92a5-4d04-b2dd-e1f5b3b6d344   
197  2e9ce8ff-084a-4c4b-9146-8cdcf7164b73   
198  388c9751-9c83-49ae-b7b0-2805cf0df860   

                               id_cliente producto  cantidad  precio  \
0    f94d8fcb-dba3-496f-ba32-3f6990dbfad6     deep         4  454.25   
1    fff9f585-0d55-4b61-8a17-5dfebfc00dc8     from         7  208.14   
2    a6348e78-4d5c-45c7-b379-cb1ee8cda0cc   parent         4  471.71   
3    39b86bb2-b596-4a7c-af4a-fa885299e505    stock        19   84.80   
4    b0946d2a-2aa9-4b43-ad15-f16fb7b8c1a5   person        10  405.02   
..                         

In [11]:
# Compras 2022

# Convertir la columna 'fecha_compra' a tipo datetime si aún no se ha hecho
compras_df['fecha_compra'] = pd.to_datetime(compras_df['fecha_compra'])

# Filtrar compras de diciembre de 2022
fecha_inicio_dic_2022 = pd.Timestamp(year=2022, month=12, day=1)
fecha_fin_dic_2022 = pd.Timestamp(year=2022, month=12, day=31)

compras_2022_navidad = compras_df[(compras_df['fecha_compra'] >= fecha_inicio_dic_2022) & (compras_df['fecha_compra'] <= fecha_fin_dic_2022)]

# Mostrar el resultado
print(compras_2022_navidad)

                                id_compra  \
31   36fce0ac-b469-4db3-8123-a348638abdf6   
118  ce9e9ebd-55c3-4853-8783-e43ce65fad87   
146  516c3b0a-5385-4918-bc40-15fad4f76322   
173  cf89569d-e550-4736-8780-6e713811b519   
175  28dd2bfd-f956-4e95-bb96-4c74ffe62994   

                               id_cliente  producto  cantidad  precio  \
31   80913ac1-0463-450e-8964-6b96fa3c1628  describe        10  132.68   
118  3f07f814-91d6-4f78-a3e9-de99f10c718b     laugh         9  424.34   
146  48bddb3e-62f5-4f2b-adaa-44ca8ed6ad5e     local        18   87.28   
173  6aa56e6d-9277-4574-8457-6cc335f789bc     start         7  337.43   
175  3f07f814-91d6-4f78-a3e9-de99f10c718b     black         1  278.19   

    fecha_compra  
31    2022-12-26  
118   2022-12-21  
146   2022-12-21  
173   2022-12-28  
175   2022-12-04  


In [12]:
# Compras concatenadas

# Concatenar ambos dataframes
compras_concatenadas = pd.concat([compras_2023_hoy, compras_2022_navidad], ignore_index=True)

# Mostrar el resultado
print(compras_concatenadas)

                                id_compra  \
0    b1fead13-f280-4f1d-9a6c-0db952171c88   
1    f7fe604f-65f3-4779-b6ee-5cdb2d79a2d1   
2    f7bf5244-7db2-47aa-a205-e54fb1e8c09d   
3    d105c8d6-47c1-4017-a045-2909536d47d7   
4    33261887-9058-41df-ae9b-f3e11d27c09a   
..                                    ...   
136  36fce0ac-b469-4db3-8123-a348638abdf6   
137  ce9e9ebd-55c3-4853-8783-e43ce65fad87   
138  516c3b0a-5385-4918-bc40-15fad4f76322   
139  cf89569d-e550-4736-8780-6e713811b519   
140  28dd2bfd-f956-4e95-bb96-4c74ffe62994   

                               id_cliente  producto  cantidad  precio  \
0    f94d8fcb-dba3-496f-ba32-3f6990dbfad6      deep         4  454.25   
1    fff9f585-0d55-4b61-8a17-5dfebfc00dc8      from         7  208.14   
2    a6348e78-4d5c-45c7-b379-cb1ee8cda0cc    parent         4  471.71   
3    39b86bb2-b596-4a7c-af4a-fa885299e505     stock        19   84.80   
4    b0946d2a-2aa9-4b43-ad15-f16fb7b8c1a5    person        10  405.02   
..                   

## 4. Join

Realice un inner join de las dos bases de datos sobre el campo `id_cliente` y guárdelo en un dataframe llamado `df_completo`. (2 puntos)

In [13]:
# Join

# Realizar un inner join en el campo 'id_cliente'
df_completo = pd.merge(clientes_df, compras_df, on='id_cliente', how='inner')

# Mostrar el resultado
print(df_completo.head())

                             id_cliente        nombre  \
0  bdd640fb-0667-4ad1-9c80-317fa3b1799d  Daniel Doyle   
1  bdd640fb-0667-4ad1-9c80-317fa3b1799d  Daniel Doyle   
2  bdd640fb-0667-4ad1-9c80-317fa3b1799d  Daniel Doyle   
3  bdd640fb-0667-4ad1-9c80-317fa3b1799d  Daniel Doyle   
4  bdd640fb-0667-4ad1-9c80-317fa3b1799d  Daniel Doyle   

                                       direccion                  email  edad  \
0  600 Jeffery Parkways\nNew Jamesside, MT 29394  zlawrence@example.org  56.0   
1  600 Jeffery Parkways\nNew Jamesside, MT 29394  zlawrence@example.org  56.0   
2  600 Jeffery Parkways\nNew Jamesside, MT 29394  zlawrence@example.org  56.0   
3  600 Jeffery Parkways\nNew Jamesside, MT 29394  zlawrence@example.org  56.0   
4  600 Jeffery Parkways\nNew Jamesside, MT 29394  zlawrence@example.org  56.0   

  fecha_registro  activo                             id_compra  producto  \
0     2023-03-23   False  23a13e20-7fd2-45ee-ac48-436668bd80d7  property   
1     2023-03-23  

## 5. Filtros

Halle los clientes activos que hayan realizado compras durante el año 2023. Guárdelo en un `.csv` llamado `clientes_activos_2023.csv`. (3 puntos)

Halle los clientes que hayan realizado más de 3 compras y cuyo gasto total en todas las compras supere los $1000. Guárdelo en un `.csv` llamado `clientes_vip.csv`. (3 puntos)

Realice los ejercicios en celdas separadas y comente su código adecuadamente.

In [15]:
# Clientes activos

# Convertir la columna 'fecha_compra' a tipo datetime si aún no se ha hecho
df_completo['fecha_compra'] = pd.to_datetime(df_completo['fecha_compra'])

# Filtrar por clientes activos y compras realizadas durante 2023
clientes_activos_2023 = df_completo[(df_completo['activo'] == True) &
                                    (df_completo['fecha_compra'].dt.year == 2023)]

# Guardar el resultado en un archivo CSV
clientes_activos_2023.to_csv('/content/clientes_activos_2023.csv', index=False)

# Confirmar que el archivo ha sido guardado
print("Archivo clientes_activos_2023.csv guardado exitosamente.")

Archivo clientes_activos_2023.csv guardado exitosamente.


In [16]:
# Clientes vip

# Agrupar por id_cliente y calcular el número total de compras y el gasto total
agrupado = df_completo.groupby('id_cliente').agg(
    numero_compras=pd.NamedAgg(column='id_compra', aggfunc='count'),
    gasto_total=pd.NamedAgg(column='precio', aggfunc='sum')
)

# Filtrar clientes con más de 3 compras y un gasto total superior a $1000
clientes_vip = agrupado[(agrupado['numero_compras'] > 3) & (agrupado['gasto_total'] > 1000)]

# Para obtener los detalles completos de estos clientes, realizar un merge con la información original de clientes
clientes_vip_completo = pd.merge(clientes_vip, clientes_df, on='id_cliente', how='inner')

# Guardar el resultado en un archivo CSV
clientes_vip_completo.to_csv('/content/clientes_vip.csv', index=False)

# Confirmar que el archivo ha sido guardado
print("Archivo clientes_vip.csv guardado exitosamente.")

Archivo clientes_vip.csv guardado exitosamente.
